#원서에서는 Ch17에 별도의 ipynb 노트북 파일을 제공하지 않습니다.
#여기서는 독자 여러분의 편의를 위해 Ch17에서 언급한 코드를 입력해 두었습니다.

In [ ]:
### 이하 Ch17의 본문에서 언급된 코드들입니다.
### 서로 연결되어 실행되지 않습니다.
### 단순하게 코드 자체만 참조하시기 바랍니다.

special_tokens_dict = {'bos_token': '<BOS>', 'eos_token': '<EOS>', 'pad_token': '<PAD>'}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)


In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 예제 문장
sentences = ["Hello world!", "Attention masks are important."]
encoded_input = tokenizer(sentences, padding='max_length', truncation=True, max_length=10, return_attention_mask=True)

print(encoded_input['input_ids'])
print(encoded_input['attention_mask'])


In [ ]:
data_collator = DataCollatorWithPading(tokenizer=tokenizer)

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from torch.utils.data import Dataset

# 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 데이터 준비
sentences = ["Hello world!", "I love machine learning.", "Transformers are powerful.", "HuggingFace is great for NLP tasks."]
labels = [0, 1, 1, 0]

# 패딩 및 텐서로 변환 조치 없이 토큰화
encodings = tokenizer(sentences, truncation=True, padding=False, return_tensors=None)

# CustomDataset 함수 정의
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
      return len(self.labels)

dataset = CustomDataset(encodings, labels)

# 모델 초기화
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# 동적 패딩을 위한 Data Collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Training Arguments
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    logging_dir='./logs',
    logging_steps=1,
    evaluation_strategy="steps",
    eval_steps=1,
    save_strategy="steps",
    save_steps=1,
    no_cuda=False,
    output_dir="./results",
    overwrite_output_dir=True,
    do_train=True
)

# Trainer 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator
)

# 학습
trainer.train()


In [ ]:
print('dataset[0]',dataset[0]['input_ids'])
print('dataset[1]',dataset[1]['input_ids'])


In [ ]:
# 포워드 패스
output = model(input_tensor)
loss = loss_fn(output, target_tensor)

# 백워드 패스
optimizer.zero_grad()
loss.backward()

# 기울기 클리핑(Gradient Clipping)
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

# 옵티마이저 step
optimizer.step()



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
assert tensor.device == next(model.parameters()).device, "Tensor and model parameters must be on the same device"